In [1]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import pandas as pd
import numpy as np

dataset = pd.DataFrame({
    'City': ['Berlin', 'Dakar', 'Kananga'],
    'State': ['Berlin', 'Dakar', 'Kasai-Occidental'],
    'Country': ['Germany', 'Senegal', 'Democratic Republic of the Congo'],
    'Lon': [np.nan, np.nan, np.nan],
    'Lat': [np.nan, np.nan, np.nan]
})

geolocator = Nominatim(user_agent='PowerBIGography')
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1, error_wait_seconds=5, max_retries=2)

def get_info(x):
    try:
        # if pd.notnull(x['Lat']) and pd.notnull(x['Lon']):
        #     return x['Lat'], x['Lon']
        global geocode
        # address = x['City'] + ', ' + x['State'] + ', ' + x['Country']
        address = x['City'] + ', ' + x['Country']
        location = geocode(address)
        if location:
            return location.latitude, location.longitude
        else:
            return np.nan, np.nan
    except:
        return np.nan, np.nan
    
    
# dataset[['Lat', 'Lon']] = dataset.apply(get_info, axis=1, result_type='expand')

In [2]:
dataset

,City,State,Country,Lon,Lat
0,Berlin,Berlin,Germany,NaN,NaN
1,Dakar,Dakar,Senegal,NaN,NaN
2,Kananga,Kasai-Occidental,Democratic Republic of the Congo,NaN,NaN


In [3]:
geo = pd.read_csv('dim_geo.csv')
geo[['Lat', 'Lon']] = geo.apply(get_info, axis=1, result_type='expand')

RateLimiter caught an error, retrying (0/2 tries). Called with (*('San Gabriel, United States',), **{}).
Traceback (most recent call last):
  File "C:\Users\Arash\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\urllib3\connectionpool.py", line 534, in _make_request
    response = conn.getresponse()
  File "C:\Users\Arash\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\urllib3\connection.py", line 565, in getresponse
    httplib_response = super().getresponse()
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\http\client.py", line 1377, in getresponse
    response.begin()
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\http\client.py", line 320, in begin
    version, status, reason = self._read_status()
  File "C:\Program File

In [13]:
def get_info_not_null(x):
    try:
        if pd.notnull(x['Lat']) and pd.notnull(x['Lon']):
            return x['Lat'], x['Lon']
        global geocode
        address = x['City'] + ', ' + x['State'] + ', ' + x['Country']
        # address = x['City'] + ', ' + x['Country']
        location = geocode(address)
        if location:
            return location.latitude, location.longitude
        else:
            return np.nan, np.nan
    except:
        return np.nan, np.nan

In [14]:
geo[['Lat', 'Lon']] = geo.apply(get_info_not_null, axis=1, result_type='expand')

RateLimiter caught an error, retrying (0/2 tries). Called with (*("Wadi as Sir, 'Amman, Jordan",), **{}).
Traceback (most recent call last):
  File "C:\Users\Arash\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\urllib3\connectionpool.py", line 534, in _make_request
    response = conn.getresponse()
  File "C:\Users\Arash\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\urllib3\connection.py", line 565, in getresponse
    httplib_response = super().getresponse()
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\http\client.py", line 1377, in getresponse
    response.begin()
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\http\client.py", line 320, in begin
    version, status, reason = self._read_status()
  File "C:\Program Fil

In [15]:
geo.isna().sum()

Geo_id     0
City       0
State      0
Region     0
Country    0
Lat        9
Lon        9
dtype: int64

In [12]:
geo[geo['Lat'].isna()]

,Geo_id,City,State,Region,Country,Lat,Lon
744,745,Kryvyy Rih,Dnipropetrovs'k,EMEA,Ukraine,NaN,NaN
927,928,Bene Beraq,Tel Aviv,EMEA,Israel,NaN,NaN
1953,1954,Berdyans'k,Zaporizhzhya,EMEA,Ukraine,NaN,NaN
1968,1969,Thies Nones,Thies,Africa,Senegal,NaN,NaN
1990,1991,Wadi as Sir,'Amman,EMEA,Jordan,NaN,NaN
2047,2048,K'ut'aisi,Imereti,EMEA,Georgia,NaN,NaN
2840,2841,Dzhezkazgan,Qaraghandy,EMEA,Kazakhstan,NaN,NaN
2918,2919,Fresnillo de González Echeverría,Zacatecas,North,Mexico,NaN,NaN
3386,3387,Kafr ad Dawwar,Al Buhayrah,Africa,Egypt,NaN,NaN


In [17]:
geo.to_csv('dim_geo_complete.csv', index=False)

In [18]:
g = pd.read_csv('dim_geo_complete.csv')